<a href="https://colab.research.google.com/github/piroo8/Aps360/blob/nicks-stuff/DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=b55d7c8e1790d4b8cc4d2161293a5cb2fedd7b1f8177717c8e3d308d5e454a3a
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


In [ ]:
!pip install nltk

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import emoji

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.util import mark_negation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import contractions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [137]:
headers = ["id", "company", "sentiment", "tweet"]
df_train = pd.read_csv('/content/sample_data/twitter_training.csv', names=headers)
df_val = pd.read_csv('/content/sample_data/twitter_validation.csv', names=headers)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [138]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [136]:
# Step 1: Remove NaN values
def remove_nan_values(df, columns):
    for column in columns:
        df = df[df[column].notna()]  # Remove rows with NaN values
        df = df[df[column].astype(bool)]  # Remove rows with empty lists or lists with only white spaces
    return df

# Step 2: Assign tweet labels
def one_hot_encode_sentiments(sentiment):
    if sentiment == 'Positive':
        return 0
    elif sentiment == 'Negative':
        return 1
    elif sentiment == 'Neutral':
        return 2
    elif sentiment == 'Irrelevant':
        return 3
    else:
      return '?><?'

def expand_slang(text):
    tokens = word_tokenize(text)
    tokens = [abbreviations.get(word.lower(), word) for word in tokens]
    text = ' '.join(tokens)
    return text


# Step 3: Clean tweets
def clean_tweets(tweet):
    # Remove @mentions
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)

    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)

    # Convert emojis to text
    tweet = emoji.demojize(tweet)

    # Expand contractions
    tweet = contractions.fix(tweet)
    ##############################3

    # Remove numbers
    tweet = re.sub(r'\d+', '', tweet)

    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove excessive whitespace
    tweet = re.sub(r'\s+', ' ', tweet)

    # Standardize repeated characters
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)

    #Expand slang terms
    tweet = expand_slang(tweet)

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Lowercase and normalize
    tokens = [token.lower() for token in tokens]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Handle abbreviations and acronyms
    abbreviation_words = {"lol": "laughing out loud", "btw": "by the way"}  # Add more as needed
    tokens = [abbreviation_words.get(token.lower(), token) for token in tokens]

    # Handle negation using mark_negation
    #negation_tokens = mark_negation(tokens)
    #tweet = "I don't like this product. It is not good."
    #['I', "don't", 'like_NEG', 'this_NEG', 'product_NEG', '.', 'It_NEG', 'is_NEG', 'not_NEG', 'good_NEG', '.']

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Uncomment the following lines to include POS tags as comments
    #pos_tags = nltk.pos_tag(tokens)
    #tokens_with_pos = [f"{token}/{pos}" for token, pos in pos_tags]

    return tokens

In [139]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

     id      company sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company   sentiment  \
0  3364   Facebook  Irrelevant   
1   352     Amazon     Neutral   
2  8312  Microsoft    Negative   
3  4371      CS-GO    Negative   
4  4433     Google     Neutral   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [140]:
# Step 2: Assign tweet labels
df_train['sentiment'] = df_train['sentiment'].apply(one_hot_encode_sentiments)
df_val['sentiment'] = df_val['sentiment'].apply(one_hot_encode_sentiments)

print(df_train.head())
print(df_val.head())

     id      company  sentiment  \
0  2401  Borderlands          0   
1  2401  Borderlands          0   
2  2401  Borderlands          0   
3  2401  Borderlands          0   
4  2401  Borderlands          0   

                                               tweet  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
     id    company  sentiment  \
0  3364   Facebook          3   
1   352     Amazon          2   
2  8312  Microsoft          1   
3  4371      CS-GO          1   
4  4433     Google          2   

                                               tweet  
0  I mentioned on Facebook that I was struggling ...  
1  BBC News - Amazon boss Jeff Bezos rejects clai...  
2  @Microsoft Why do I pay for WORD when it funct...  
3  CSGO matchmaking is so full of closet

In [141]:
# Step 3: Clean tweets
df_train['tweet'] = df_train['tweet'].apply(clean_tweets)
df_val['tweet'] = df_val['tweet'].apply(clean_tweets)

print(df_train.head())
print(df_val.head())

     id      company  sentiment                          tweet
0  2401  Borderlands          0  [getting, borderland, murder]
1  2401  Borderlands          0         [coming, border, kill]
2  2401  Borderlands          0    [getting, borderland, kill]
3  2401  Borderlands          0   [coming, borderland, murder]
4  2401  Borderlands          0  [getting, borderland, murder]
     id    company  sentiment  \
0  3364   Facebook          3   
1   352     Amazon          2   
2  8312  Microsoft          1   
3  4371      CS-GO          1   
4  4433     Google          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full, closet, hacking, tru...  
4  [president, slapping, american, face, really, ...  


In [142]:
import pandas as pd
from collections import Counter

# Assuming you have a DataFrame named df_train with columns 'sentiment' and 'tweet'

# Step 1: Collect the tweets and sentiment encodings from the DataFrame
tweets = df_train['tweet'].tolist()
sentiments = df_train['sentiment'].tolist()

# Step 3: Combine the tweets with their corresponding sentiment encodings into a list of tuples
tweet_sentiment_pairs = list(zip(tweets, sentiments))

# Step 4: Count the frequency of each word for each sentiment
word_frequencies = {
    tuple([1, 0, 0, 0]): Counter(),
    tuple([0, 1, 0, 0]): Counter(),
    tuple([0, 0, 1, 0]): Counter(),
    tuple([0, 0, 0, 1]): Counter()
}

for tweet, sentiment in tweet_sentiment_pairs:
    word_frequencies[tuple(sentiment)].update(tweet)

# Step 5: Determine the most frequent words for each sentiment
most_frequent_words = {
    tuple([1, 0, 0, 0]): [],
    tuple([0, 1, 0, 0]): [],
    tuple([0, 0, 1, 0]): [],
    tuple([0, 0, 0, 1]): []
}
infrequent_words = set()

for sentiment, counter in word_frequencies.items():
    most_common_words = counter.most_common()
    most_frequent_words[sentiment] = most_common_words

    # Step 6: Handle infrequent words
    frequency_threshold = 5  # Adjust this threshold as needed
    infrequent_words.update(word for word, frequency in most_common_words if frequency <= frequency_threshold)

# Print the most frequent words for each sentiment
sentiment_mapping = {
    tuple([1, 0, 0, 0]): 'Positive',
    tuple([0, 1, 0, 0]): 'Negative',
    tuple([0, 0, 1, 0]): 'Neutral',
    tuple([0, 0, 0, 1]): 'Irrelevant'
}

for sentiment, words in most_frequent_words.items():
    print(f"Most frequent words for sentiment {sentiment_mapping[sentiment]}:")
    for word, frequency in words:
        print(f"{word}: {frequency}")
    print()

# Example of handling infrequent words
print("Infrequent words:")
print(infrequent_words)

TypeError: ignored

In [143]:
# Step 1: Remove NaN values
columns_to_check = ["id", "company", "sentiment", "tweet"]
df_train = remove_nan_values(df_train, columns_to_check)
df_val = remove_nan_values(df_val, columns_to_check)

print(df_train.head())
print(df_val.head())

      id      company  sentiment  \
12  2403  Borderlands          2   
13  2403  Borderlands          2   
14  2403  Borderlands          2   
15  2403  Borderlands          2   
16  2403  Borderlands          2   

                                                tweet  
12  [rockhard, la, varlope, rare, powerful, handso...  
13  [rockhard, la, varlope, rare, powerful, handso...  
14  [rockhard, la, varlope, rare, powerful, handso...  
15  [rockhard, la, vita, rare, powerful, handsome,...  
16  [live, rock, hard, music, la, la, varlope, rar...  
     id    company  sentiment  \
0  3364   Facebook          3   
1   352     Amazon          2   
2  8312  Microsoft          1   
3  4371      CS-GO          1   
4  4433     Google          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full,

In [144]:
# Print the resulting DataFrame
print(df_train.head())
print(df_val.head())

      id      company  sentiment  \
12  2403  Borderlands          2   
13  2403  Borderlands          2   
14  2403  Borderlands          2   
15  2403  Borderlands          2   
16  2403  Borderlands          2   

                                                tweet  
12  [rockhard, la, varlope, rare, powerful, handso...  
13  [rockhard, la, varlope, rare, powerful, handso...  
14  [rockhard, la, varlope, rare, powerful, handso...  
15  [rockhard, la, vita, rare, powerful, handsome,...  
16  [live, rock, hard, music, la, la, varlope, rar...  
     id    company  sentiment  \
0  3364   Facebook          3   
1   352     Amazon          2   
2  8312  Microsoft          1   
3  4371      CS-GO          1   
4  4433     Google          2   

                                               tweet  
0  [mentioned, facebook, struggling, motivation, ...  
1  [british, broadcasting, corporation, news, ama...  
2  [pay, word, function, poorly, chromebook, face...  
3  [csgo, matchmaking, full,

In [145]:
# Assuming your DataFrame is named df
import torch
# Drop the 'id' column
df_train_new = df_train.drop('id', axis=1)
df_val_new = df_val.drop('id', axis=1)

# Export the DataFrame to a CSV file
df_train_new.to_csv('df_train_new_int.csv', index=False)
df_val_new.to_csv('df_val_new_int.csv', index=False)

#create dataloaders
train_loader = torch.utils.data.DataLoader(df_train_new, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(df_val_new, batch_size=64, shuffle=True)

In [146]:
print(df_train_new[:5])

        company  sentiment                                              tweet
12  Borderlands          2  [rockhard, la, varlope, rare, powerful, handso...
13  Borderlands          2  [rockhard, la, varlope, rare, powerful, handso...
14  Borderlands          2  [rockhard, la, varlope, rare, powerful, handso...
15  Borderlands          2  [rockhard, la, vita, rare, powerful, handsome,...
16  Borderlands          2  [live, rock, hard, music, la, la, varlope, rar...


In [147]:
import torch
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
# Assuming you have a series of lists in your DataFrame column 'tweet'
# For example, assuming your DataFrame is named 'df_train_new' and 'df_val_new', and the column with lists is 'tweet'
series_of_lists_train = df_train_new['tweet']
series_of_lists_valid = df_val_new['tweet']

# Step 1: Flatten the series of lists into a single list for both training and validation sets
all_words = [word for sublist in series_of_lists_train for word in sublist]
all_words_valid = [word for sublist in series_of_lists_valid for word in sublist]

# Combine the words from both training and validation sets
all_words.extend(all_words_valid)

# Step 2: Count the occurrences of each word
word_counts = Counter(all_words)

# Step 3: Sort the words based on their frequency in descending order
sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Step 4: Create a vocabulary dictionary with words ordered by frequency
vocab_dict = {word: idx for idx, (word, _) in enumerate(sorted_words)}

# Step 5: Convert the series of lists into sequences of indices using the vocabulary dictionary
sequences_of_indices_train = [[vocab_dict[word] for word in sublist] for sublist in series_of_lists_train]
sequences_of_indices_valid = [[vocab_dict[word] for word in sublist] for sublist in series_of_lists_valid]

# Step 6: Pad the sequences with zeros to make them all the same length
# Find the maximum length of a sequence in both training and validation sets combined
max_seq_length_train = max(len(seq) for seq in sequences_of_indices_train)
max_seq_length_valid = max(len(seq) for seq in sequences_of_indices_valid)
max_seq_length = max(max_seq_length_train, max_seq_length_valid)

padded_sequences_train = pad_sequence([torch.tensor(seq[:max_seq_length]) for seq in sequences_of_indices_train], batch_first=True, padding_value=0)
padded_sequences_valid = pad_sequence([torch.tensor(seq + [0] * (max_seq_length - len(seq))) for seq in sequences_of_indices_valid], batch_first=True, padding_value=0)


# Ensure the shapes of both padded sequences
print(padded_sequences_train.shape)
print(padded_sequences_valid.shape)


torch.Size([52141, 198])
torch.Size([722, 198])


In [148]:
# train_labels_list = torch.tensor(df_train_new['sentiment'])
# val_labels_list = torch.tensor(df_val_new['sentiment'])
train_label_list = []
valid_label_list = []
for label in df_train_new['sentiment']:
  train_label_list.append(label)
for label in df_val_new['sentiment']:
  valid_label_list.append(label)
train_labels_tensor = torch.tensor(train_label_list)
valid_labels_tensor = torch.tensor(valid_label_list)

In [149]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, sequences, labels, h0_c0=None):
        self.sequences = sequences
        self.labels = labels
        self.h0_c0 = h0_c0

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]

        if self.h0_c0 is not None:
            h0_c0 = self.h0_c0[idx]
            return sequence, label, h0_c0
        else:
            return sequence, label, None

def collate_fn(batch):
    # Separate the elements of the batch into individual lists
    sequences, labels, h0_c0 = zip(*batch)

    # Pad the sequences with zeros to make them the same length
    padded_sequences = pad_sequence(sequences, batch_first=True)

    # Stack the tensors
    labels = torch.tensor(labels)

    # Filter out None values in h0_c0
    h0_c0 = [tensor for tensor in h0_c0 if tensor is not None]

    if h0_c0:
        # Separate h0 and c0 tensors
        h0 = torch.stack([tensor[0] for tensor in h0_c0])
        c0 = torch.stack([tensor[1] for tensor in h0_c0])
    else:
        # If h0_c0 is empty, create tensors of zeros with appropriate shapes
        batch_size = padded_sequences.size(0)
        num_layers = 1  # Set the appropriate num_layers
        hidden_size = 128  # Set the appropriate hidden_size

        h0 = torch.zeros(num_layers, batch_size, hidden_size).to(padded_sequences.device)
        c0 = torch.zeros(num_layers, batch_size, hidden_size).to(padded_sequences.device)

    return padded_sequences, labels, h0, c0
# ... (your data preprocessing steps) ...

# Convert sequences and labels to torch.Tensor
sequences_tensor = [torch.tensor(seq) for seq in sequences_of_indices_train]
labels_tensor = torch.tensor(train_label_list)

# Create custom dataset and data loader with collate_fn
train_dataset = CustomDataset(sequences_tensor, labels_tensor)
batch_size = 32
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [150]:
prepared_train_dataset = torch.utils.data.TensorDataset(padded_sequences_train, train_labels_tensor)
prepared_valid_dataset = torch.utils.data.TensorDataset(padded_sequences_valid, valid_labels_tensor)

train_data_loader = torch.utils.data.DataLoader(prepared_train_dataset, batch_size=32, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(prepared_valid_dataset, batch_size=32, shuffle=True)

In [151]:
for batch_data, batch_labels in valid_data_loader:
    print("Data:", batch_data)
    print("Labels:", batch_labels)

Data: tensor([[   51,    70,    10,  ...,     0,     0,     0],
        [  427,  1793,   302,  ...,     0,     0,     0],
        [  177,     2,    79,  ...,     0,     0,     0],
        ...,
        [ 1660,     6, 10452,  ...,     0,     0,     0],
        [29374,  1296,   174,  ...,     0,     0,     0],
        [10113, 10114,  1311,  ...,     0,     0,     0]])
Labels: tensor([3, 1, 2, 2, 3, 3, 3, 2, 3, 1, 1, 1, 1, 1, 2, 2, 3, 3, 1, 1, 1, 1, 1, 2,
        1, 2, 3, 2, 3, 3, 2, 3])
Data: tensor([[  91,  180,   20,  ...,    0,    0,    0],
        [1011, 7326,   47,  ...,    0,    0,    0],
        [ 283,   18, 1344,  ...,    0,    0,    0],
        ...,
        [2671,  829,    0,  ...,    0,    0,    0],
        [ 286,  125, 3615,  ...,    0,    0,    0],
        [  11,    2,  542,  ...,    0,    0,    0]])
Labels: tensor([1, 2, 3, 3, 2, 1, 1, 2, 3, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2,
        2, 1, 1, 2, 2, 3, 1, 3])
Data: tensor([[2865,   84,  666,  ...,    0,    0,    0],


In [152]:
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.to(torch.float32)

    def forward(self, x, h0=None, c0=None):
        # If the initial hidden and cell states are not provided, initialize them with zeros
        if h0 is None:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(x.dtype)
        if c0 is None:
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(x.dtype)

        # Add an additional dimension for sequence_length
        x = x.unsqueeze(1)

        # Pass the input through the LSTM layer
        out, _ = self.rnn(x, (h0, c0))

        # Extract the output of the last time step
        out = out[:, -1, :]

        # Pass the output through the fully connected layer for sentiment prediction
        out = self.fc(out)

        return out


def train(model, data_loader, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Check if GPU is available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode

        total_loss = 0.0
        # for data, labels in data_loader:
        #     print(data)
        #     print(labels)
        #     # data, labels = data.to(device), labels.to(device)

        #     # # Forward pass
        #     predictions = model(data)
        #     print(predictions)
            # # Compute the loss
            # loss = criterion(predictions, labels)

            # # Backpropagation and optimization
            # optimizer.zero_grad()
            # loss.backward()
            # optimizer.step()

            # total_loss += loss.item()

        # Calculate average loss for the epoch
        # average_loss = total_loss / len(data_loader)

        # print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}")
        for data, labels in data_loader:
            data, labels = data.to(device), labels.float().to(device)

            # Initialize the hidden and cell states for each batch
            h0 = torch.zeros(model.num_layers, data.size(0), model.hidden_size).to(device).to(data.dtype)
            c0 = torch.zeros(model.num_layers, data.size(0), model.hidden_size).to(device).to(data.dtype)

            # Forward pass
            predictions = model(data, h0, c0)

            # Compute the loss
            loss = criterion(predictions, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()



In [153]:
import torch
import torch.nn as nn
import torch.optim as optim

class SentimentRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SentimentRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define the fully connected layer for sentiment prediction
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
      # Initialize hidden and cell states
      h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(self.lstm.weight_ih_l0.dtype)
      c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).to(self.lstm.weight_ih_l0.dtype)

      # Convert input tensor to the same data type as the LSTM weights
      x = x.to(self.lstm.weight_ih_l0.dtype)
      out, _ = self.lstm(x, (h0, c0))

      # Extract the output of the last time step
      out = out[:, -1, :]

      # Pass the output through the fully connected layer for sentiment prediction
      out = self.fc(out)

      return out

def train(model, train_dataloader, learning_rate):
    device = torch.device("cpu")
    model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    epochs = 10
    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for batch in train_dataloader:
            # Move the batch to the device
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            # print(inputs)
            # print(labels)
        #     # Reshape the inputs to match the expected shape
            inputs = inputs.unsqueeze(1)  # Add an extra dimension for sequence length
            # print(inputs)
            # Clear the gradients
            optimizer.zero_grad()

        #     # Forward pass
            outputs = model(inputs)

        #     # Reshape the outputs and labels
            outputs = outputs.view(-1, outputs.size(-1))
            labels = labels.view(-1)

        #     # Convert labels to torch.long
            labels = labels.to(torch.long)
        #     # Calculate the loss
            loss = loss_fn(outputs, labels)

        #     # Backward pass and optimization
            loss.backward()
            optimizer.step()

        #     # Update the running loss
            running_loss += loss.item()

        #     # Calculate the number of correct predictions
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        # # Calculate the training accuracy
        training_accuracy = correct_predictions / total_predictions

        # # Print the average loss and accuracy for the epoch
        epoch_loss = running_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss}, Accuracy: {training_accuracy * 100}%")

In [155]:
first_model = SentimentRNN(198,64, 1, 4)
train(first_model, train_data_loader, 0.0001)

Epoch [1/10], Loss: 1.3892389701187977, Accuracy: 28.599374772252162%
Epoch [2/10], Loss: 1.3511371712011793, Accuracy: 33.401737596133565%
Epoch [3/10], Loss: 1.3239180294282598, Accuracy: 36.10210774630329%
Epoch [4/10], Loss: 1.3009230779723886, Accuracy: 37.507911240674325%
Epoch [5/10], Loss: 1.2829865894434642, Accuracy: 38.731516464969985%
Epoch [6/10], Loss: 1.2660110488991065, Accuracy: 39.44880228610882%
Epoch [7/10], Loss: 1.2506975177606923, Accuracy: 40.04909763909399%
Epoch [8/10], Loss: 1.2372837141247615, Accuracy: 40.78364434897681%
Epoch [9/10], Loss: 1.225201642220737, Accuracy: 41.360925183636674%
Epoch [10/10], Loss: 1.215578316837732, Accuracy: 41.665867551447036%


In [ ]:
len(vocab_dict)

25457